In [1]:
import numpy
from sympy import *
#Find way to enforce arity. ArgumentIndexError almost works

In [33]:
x, t, s, z, T = symbols('x t σ z T')

class dp(Function):
    """
    d_{+}(x,s,t) = (x + (s^2/2)(T-t))/(s sqrt(T-t)) 
    """
    def fdiff(self, argindex):
        x, t, s = self.args
        if argindex == 1:
            return 1/(s * sqrt(T-t))
        if argindex == 3:
            return - dm(x,t,s)/s
        
    def doit(self,deep=True, **hints):
        x , s , t = self.args
        if deep:
            x = x.doit(deep=deep, **hints)
            s = s.doit(deep=deep, **hints)
            t = t.doit(deep=deep, **hints)
        return (x + (s**2/2)*(T-t))/(s*sqrt(T-t))
    
    def _latex(self, printer):
        x , t , s = [printer._print(i) for i in self.args]
        return r"\operatorname{d_{+}}{\left(%s,%s,%s\right)}" % (x,t,s)
        
class dm(Function):
    """
    d_{+}(x,s,t) = (x - (s^2/2)(T-t))/(s sqrt(T-t)) 
    """
    def fdiff(self, argindex=1):
        x, t, s = self.args
        if argindex == 1:
            return 1/(s * sqrt(T-t))
        if argindex == 3:
            return - dp(x,t,s)/s

    def doit(self,deep=True, **hints):
        x , s , t = self.args
        if deep:
            x = x.doit(deep=deep, **hints)
            s = s.doit(deep=deep, **hints)
            t = t.doit(deep=deep, **hints)
        return (x - (s**2/2)*(T-t))/(s*sqrt(T-t))        
        
    def _latex(self, printer):
        x , t , s = [printer._print(i) for i in self.args]
        return r"\operatorname{d_{-}}{\left(%s,%s,%s\right)}" % (x,t,s)
    
assert dp(x,s,t).doit() == (x+(s**2/2)*(T-t))/(s*sqrt(T-t))  
assert dm(x,s,t).doit() == (x-(s**2/2)*(T-t))/(s*sqrt(T-t))  
assert diff(dp(x,t,s),s) == -dm(x,t,s)/s 
assert diff(dm(x,t,s),s) == -dp(x,t,s)/s 
assert diff(dp(x,t,s),x) == diff(dm(x,t,s),x) == 1/(s*sqrt(T-t))

In [3]:
class dN(Function):
    """
    N'(z) = (1/(sqrt(2 pi sigma)) e^(-z^2/2)
    """
    @classmethod
    def eval(cls,arg):
        if arg.is_zero:
            return Integer(1)
        if isinstance(arg,dm):
            x , t , s = arg.args
            return exp(x)*dN(dp(x,t,s))
        
    
    #def fdiff(self, argindex=1):
    def fdiff(self, argindex):
        return - self.args[0] * dN(self.args[0])
    
    def _latex(self,printer):
        a = printer._print(self.args[0])
        return r"N^{\prime}{\left(%s\right)}" % (a)
    
assert dN(0) == 1
assert diff(dN(z),z) == -z*dN(z)
assert dN(dm(x,t,s)) == exp(x)*dN(dp(x,t,s))

In [4]:
class N(Function):
    """
    N(z) = (1/(sqrt(2 pi sigma)) int_{-\infty}^z e^(-z^2/2) dz
    """
    @classmethod
    def eval(cls,arg):
        if arg.is_Number:
            if arg is S.NegativeInfinity:
                return Integer(0)
            elif arg is S.Infinity:
                return Integer(1)
            elif arg.is_zero:
                return Rational(1,2)
    
    def fdiff(self, argindex=1):
        return dN(self.args[0])

assert N(S.NegativeInfinity) == 0
assert N(S.Infinity) == 1
assert N(0) == 1/2
assert diff(N(z),z) == dN(z)

In [5]:
BSM = exp(x)*N(dp(x,t,s))-N(dm(x,t,s))

In [6]:
# Delta
diff(BSM,x)

exp(x)*N(dp(x, t, σ))

In [7]:
# Vega
factor(diff(BSM,s))

-(dm(x, t, σ) - dp(x, t, σ))*exp(x)*dN(dp(x, t, σ))/σ

In [9]:
# Gamma
diff(BSM,x,x)

(N(dp(x, t, σ)) + dN(dp(x, t, σ))/(σ*sqrt(T - t)))*exp(x)

In [38]:
# Vanna
assert diff(BSM,s,x) == diff(BSM,x,s)
diff(BSM,x,s)

-dm(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ

In [41]:
# Vomma
diff(diff(BSM,s),s)

TypeError: dp._latex() got an unexpected keyword argument 'exp'

-dm(x, t, σ)*dm(x, t, σ)*dp(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ**2 + dm(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ**2 + dm(x, t, σ)*dp(x, t, σ)**2*exp(x)*dN(dp(x, t, σ))/σ**2 - dm(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ**2 + dp(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ**2 - dp(x, t, σ)*exp(x)*dN(dp(x, t, σ))/σ**2